In [2]:
import pandas as pd
from renumics import spotlight

In [3]:
from EmbeddingFunctions import *
config_path = "scripts\config.yaml"
config=read_config(config_path)


embeddings_framework = config["embeddings_framework"]
embeddings_model = config["embeddings_model"]
vector_db, collection = initialize_vector_database("vector_db", "pdf_docs", "vector_db", embeddings_framework, embeddings_model)


Number of documents in the database: 67


In [4]:
import pandas as pd
response = collection.get(include=['embeddings','documents'])
print(response)

{'ids': ['champs//0', 'data//0', 'data//1', 'data//2', 'data//3', 'data//4', 'data//5', 'dexcom//0', 'dexcom//1', 'dexcom//10', 'dexcom//11', 'dexcom//12', 'dexcom//13', 'dexcom//14', 'dexcom//15', 'dexcom//16', 'dexcom//17', 'dexcom//18', 'dexcom//19', 'dexcom//2', 'dexcom//3', 'dexcom//4', 'dexcom//5', 'dexcom//6', 'dexcom//7', 'dexcom//8', 'dexcom//9', 'prost//0', 'prost//1', 'prost//10', 'prost//11', 'prost//12', 'prost//13', 'prost//14', 'prost//15', 'prost//16', 'prost//17', 'prost//18', 'prost//19', 'prost//2', 'prost//20', 'prost//21', 'prost//22', 'prost//23', 'prost//24', 'prost//25', 'prost//26', 'prost//27', 'prost//28', 'prost//29', 'prost//3', 'prost//30', 'prost//31', 'prost//32', 'prost//33', 'prost//34', 'prost//35', 'prost//36', 'prost//37', 'prost//38', 'prost//39', 'prost//4', 'prost//5', 'prost//6', 'prost//7', 'prost//8', 'prost//9'], 'embeddings': [[-0.03394703567028046, -0.31514662504196167, 0.15680938959121704, 0.38811516761779785, -0.2037956416606903, 0.279936

In [6]:
id=response["ids"]
id_pref = []
for i in id:
    pref = i.split("//")[0]
    id_pref.append(pref)
df = pd.DataFrame({
 "id": response["ids"],
 "pref": id_pref,
 "document": response['documents'],
 "embedding": response['embeddings'],
})
df.head()

,id,pref,document,embedding
0,champs//0,champs,page_content='RESEARCHVISITOR’SAGREEMENT\nAny ...,"[-0.03394703567028046, -0.31514662504196167, 0..."
1,data//0,data,page_content='- 1 - 2023-2024 Programming for...,"[0.02350587397813797, -0.2088620662689209, 0.2..."
2,data//1,data,page_content='Programming for Data Science 202...,"[0.3847985863685608, -0.1391836255788803, -0.1..."
3,data//2,data,page_content='Programming for Data Science 202...,"[0.6047042608261108, -0.40383514761924744, 0.0..."
4,data//3,data,page_content='Programming for Data Science 202...,"[0.038008563220500946, 0.014930032193660736, -..."


In [7]:
df['input'] = 'document'
df.head()

,id,pref,document,embedding,input
0,champs//0,champs,page_content='RESEARCHVISITOR’SAGREEMENT\nAny ...,"[-0.03394703567028046, -0.31514662504196167, 0...",document
1,data//0,data,page_content='- 1 - 2023-2024 Programming for...,"[0.02350587397813797, -0.2088620662689209, 0.2...",document
2,data//1,data,page_content='Programming for Data Science 202...,"[0.3847985863685608, -0.1391836255788803, -0.1...",document
3,data//2,data,page_content='Programming for Data Science 202...,"[0.6047042608261108, -0.40383514761924744, 0.0...",document
4,data//3,data,page_content='Programming for Data Science 202...,"[0.038008563220500946, 0.014930032193660736, -...",document


In [17]:
import ollama

questions = [
    "Whats the dexcom device?",
    "Whats are the guidelines for prostate cancer treatment?",
    "What is the goal of the data science project?"
]

for question in questions:
    response = ollama.embeddings(model=embeddings_model, prompt=question)
    embedding = response["embedding"]
    
    # Create a new row with the question and its embedding
    new_line = {
        'id': "None",
        'pref': "None",
        'document': question,
        'embedding': embedding,
        'input': 'question'
    }
    
    # Append the new row to the DataFrame
    df = pd.concat([df, pd.DataFrame([new_line])], ignore_index=True)

df.head()


ResponseError: failed to generate embedding

In [ ]:
from renumics import spotlight
spotlight.show(df)

In [ ]:
import umap
import matplotlib.pyplot as plt
import numpy as np
# Prepare the embeddings for UMAP
embeddings = np.array([np.array(x) for x in df["embedding"]])

# Reduce dimensionality with UMAP
reducer = umap.UMAP()
embedding_reduced = reducer.fit_transform(embeddings)

# Plot the reduced embeddings
plt.scatter(embedding_reduced[:,  0], embedding_reduced[:,  1], c='gray', alpha=0.2)

# Add labels and title
plt.xlabel("UMAP  1")
plt.ylabel("UMAP  2")
plt.show()